# Computational assigment 3
Kristine Schüller and Sigrid Aunsmo

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import scipy.constants as sc


"""Defining constants and initial values"""
q0=0.5 #default damping parameter
delta_t=0.01 #value for each timestep
theta0 = 0.2 #initial value of the angle theta
omega0 = 0 #inital value for the pendulums angular velocity 
g=sc.g #gravitational acceleration
l=9.8 #length of pendulum
Omega_d=2/3 #driving frequency
Fd=0.5 #driving force

"plotting parameters "
fontsize=15
newparams = {'axes.titlesize': fontsize, 'axes.labelsize': fontsize, 
             'figure.titlesize':20,
             'lines.linewidth': 2, 'lines.markersize': 7,
             'figure.figsize': (20, 5), 'ytick.labelsize': fontsize,
             'xtick.labelsize': fontsize, 'legend.fontsize': fontsize,
            'legend.handlelength': 1.0}

plt.rcParams.update(newparams)


In [ ]:

"""Defining functions"""
#h=skrittlengde

#Runge Kutta 
def Kutta4(f,t0,y0,h,nsteps,Fd,q=1,omega_D=1):
    Y=np.zeros((nsteps,2))  #2D array. Y[:,0] is theta values and Y[:,1] is omega values 
    Y[0]=y0 #initial values 
    T=np.linspace(t0, t0 + h*nsteps, nsteps)
    for i in range(nsteps-1): 
        t=i*h
        F1= f(Y[i],t,q,omega_D,Fd)
        F2= f(Y[i] + h/2* F1,t,q,omega_D,Fd)
        F3= f(Y[i] + h/2* F2,t,q,omega_D,Fd)
        F4= f(Y[i] + h*F3,t,q,omega_D,Fd)
        
        Y[i+1]= Y[i] + h/6*(F1+2*F2+2*F3+F4) #calculating the new theta and omega values
        
    return Y,T #return array of theta and omega values, and time list



def f_forced(w,t,q,omega_D,Fd):
    return np.array([w[1],-q*w[1]-g/l*w[0]+Fd*np.sin(omega_D*t)])

#Function to be used in Runge Kutta
#takes in array of theta and the derivative of theta
#returns array of the derivative of theta and the double derivative of theta

### Task 1:
The plots below show $\theta$ and $\omega$ as functions of time for three different values of the driving force $F_d$. For $F_d=0$ the pendulums oscillation stops after a short time. For $F_d=0.5$ and $F_d=1.2$, 

In [ ]:
Fd_list=np.array([0.0,0.5,1.2]) #values for Fd
t_max=60 #maximum value of t
nsteps=int(t_max/delta_t) #number of timesteps
y0=np.array([theta0,omega0]) #array of initial values

#function for plotting theta and omega as functions of time for different values of Fd
def plot_omega_theta(Fd_list,f_forced,y0,delta_t,nsteps,q0,Omega_d):
    for i in range(len(Fd_list)): #for each different Fd
        #calculating theta and omega using Runge Kutta
        X,T=Kutta4(f_forced,0,y0,delta_t,nsteps,Fd_list[i],q0,Omega_d)
        theta=X[:,0]
        omega=X[:,1]

        #plotting, 2 subplots with common x-axis
        fig, axs=plt.subplots(2,1,sharex=True,figsize=(15,5))
        plt.subplot(2,1,1)
        plt.ylabel(r"$\theta$")
        #title for both plots:
        plt.suptitle(r'$\theta$ and $\omega$ as functions of time for Fd='+str(Fd_list[i]))
        plt.plot(T, theta)

        plt.subplot(2,1,2)
        plt.ylabel("$\omega$")
        plt.xlabel("t")
        plt.plot(T,omega)
        plt.show()

plot_omega_theta(Fd_list,f_forced,y0,delta_t,nsteps,q0,Omega_d)
    

### Task 2
The first two plots below show $\theta$ and $\omega$ as functions of time for the initial angle displaced by 0.001 rad. 

Below are the plots for the difference between the original and the displaced value of $\theta$ as a function of time. The y-axis has a logarithmic scale. As can be seen from the plot there is a linear trend in the local maximums for $\delta \theta$. 

In [ ]:
Fds=[0.5,1.2] #values of Fd
y0disp=np.array([theta0+0.001,omega0]) #displacing the initial angle

#plotting theta and omega for the displaced initial angle
plot_omega_theta(Fds,f_forced,y0disp,delta_t,nsteps,q0,Omega_d)

#plotting the difference delta theta
for i in range(2):
    #calculating theta using runge Kutta
    X,T=Kutta4(f_forced,0,y0,delta_t,nsteps,Fds[i],q0,Omega_d)
    Xdisp,Tdisp=Kutta4(f_forced,0,y0disp,delta_t,nsteps,Fds[i],q0,Omega_d)
    D_theta=X[:,0]-Xdisp[:,0] #difference between the theta values
    
    plt.title(r'$\Delta \theta$ for Fd='+str(Fds[i]))
    plt.xlabel("t")
    plt.ylabel(r"$\Delta \theta$")
    plt.yscale("log") #for semilogarithmic plot
    plt.plot(T,D_theta)
    plt.show()

    

### Task 3

The plots below show the trajectory in phase space. The trajectory has an eliptic shape which expands in the beginging due to the driving force. The stroboscobic plot shows that after a time $\Omega_D t = 2 \pi n$, $n=0,1,2,3...$ the ratio between $\omega$ and $\theta$ seemingly stays the same. 

In [ ]:
Fds=[0.5,1.2]
t_max3=100
delta_t3 = 2*np.pi/Omega_d /1000
nsteps3=int(t_max/delta_t)

colors=["blue", "orange", "red", "green"]

plt.figure(figsize=(12,7))
for i in range(2):
    X,T=Kutta4(f_forced,0,y0,delta_t3,nsteps3,Fds[i],q0,Omega_d)
    theta=X[:,0]
    omega=X[:,1]
    
    #phase space plot
    plt.plot(theta,omega,color=colors[i*2],label=str(Fds[i]))
    
    #Stroboscobic plot
    theta_strob = [theta[i] for i in range(0,len(theta)-1,1000)]
    omega_strob = [omega[i] for i in range(0,len(omega)-1,1000)]
    plt.plot(theta_strob,omega_strob,'x',color=colors[i*2+1],label='stroboscobic plot')

plt.title("Trajectory and stoboscobic plot in phase space")
plt.xlabel(r'$\theta$')
plt.ylabel(r'$\omega$')
plt.legend()
plt.show()